In [34]:
import json
import numpy as np
import imageio
import pdb
from pathlib import Path
import cv2
from PIL import Image

# @markdown The base directory for all captures. This can be anything if you're running this notebook on your own Jupyter runtime.
save_dir = './data/'  
capture_name = 'nvidia/Umbrella' 
# The root directory for this capture.
root_dir = Path(save_dir, capture_name)
# Where to save RGB images.
# rgb_dir = root_dir / 'rgb'
# rgb_raw_dir = root_dir / 'images'

# make dirs for freeze test if available
freeze_test_dir = root_dir / 'freeze_test'
if freeze_test_dir.exists():
    rgb_raw_freeze_test_dir = freeze_test_dir / 'rgb' / '1x'
    rgb_freeze_test_dir = freeze_test_dir / 'rgb' / '1x'

    rgb_freeze_test_dir.mkdir(exist_ok=True, parents=True)

# tmp_rgb_raw_dir = rgb_raw_dir

# rgb_list = list(sorted(rgb_raw_dir.glob( '*')))
# image_size = imageio.imread(rgb_list[0]).shape[:2]
# rgb_list[:3]


## Write dataset split

In [35]:
# @title Save dataset split to `dataset.json`.

# write meta info for freeze test if exists
if freeze_test_dir.exists():
    all_ids_freeze_test = [img.stem for img in list(sorted(rgb_raw_freeze_test_dir.glob('*.png')))]
    train_ids_freeze_test = []
    val_ids_freeze_test = all_ids_freeze_test
    dataset_json = {
        'count': len(all_ids_freeze_test),
        'num_exemplars': len(train_ids_freeze_test),
        'ids': all_ids_freeze_test,
        'train_ids': train_ids_freeze_test,
        'val_ids': val_ids_freeze_test,
    }

    dataset_json_path = freeze_test_dir / 'dataset.json'
    with dataset_json_path.open('w') as f:
        json.dump(dataset_json, f, indent=2)

## Save metadata info

In [36]:
# @title Save metadata information to `metadata.json`.
import bisect



# write meta info for freeze test if exists
if freeze_test_dir.exists():
  freeze_frame = {
      'warp_id': 12,
      'appearance_id': 12,
      'camera_id': 0,
  }
  # freeze_frame = metadata_json[list(sorted(rgb_raw_freeze_test_dir.glob('*.png')))[k_freeze_meta['test_freeze_frame']].stem]

  metadata_freeze_json = {}
  for i, image_path in enumerate(list(sorted(rgb_raw_freeze_test_dir.glob('*.png')))):
    image_id = image_path.stem
    metadata_freeze_json[image_id] = freeze_frame

  metadata_json_path = freeze_test_dir / 'metadata.json'
  with metadata_json_path.open('w') as f:
      json.dump(metadata_freeze_json, f, indent=2)

  print(f'Saved freeze metadata information to {metadata_json_path}')


Saved freeze metadata information to data/nvidia/Umbrella/freeze_test/metadata.json


## Write scene info

In [37]:
def get_bbox_corners(points):
  lower = points.min(axis=0)
  upper = points.max(axis=0)
  return np.stack([lower, upper])

bbox_corners = get_bbox_corners(np.array(k_meta['instances'][0]['bboxes_3d'])[0])
bbox_corners = np.array([[0,   0,  0],
                         [50, 50, 10]])

# scene_center = np.mean(bbox_corners, axis=0)
scene_scale = 1.0 / np.sqrt(np.sum((bbox_corners[1] - bbox_corners[0]) ** 2))
scene_center = np.array([0., 0., 0.])
# scene_scale = 0.01

print(f'Scene Center: {scene_center}')
print(f'Scene Scale: {scene_scale}')

# @title Save scene information to `scene.json`.

scene_json_path = root_dir /  'scene_gt.json'
with scene_json_path.open('w') as f:
  json.dump({
      'scale': scene_scale,
      'center': scene_center.tolist(),
      'bbox': bbox_corners.tolist(),
      'near': 0.1,
      'far': 1. # bbox_corners[1].max() * scene_scale,
  }, f, indent=2)

print(f'Saved scene information to {scene_json_path}')

NameError: name 'k_meta' is not defined